In [1]:
%%bash
module list

bash: line 1: module: command not found


CalledProcessError: Command 'b'module list\n'' returned non-zero exit status 127.

In [1]:
%matplotlib inline

In [2]:
import os
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda

In [3]:
import aimmd
import aimmd.distributed as aimmdd

Tensorflow/Keras not available


# GMX engine

In [4]:
n_engines = 4

In [5]:
#scratch_dir = "."
#scratch_dir = "/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/"
scratch_dir = "/home/think/scratch/aimmd_distributed/"
wdirs = [os.path.join(scratch_dir, f"engine_wdir{i}") for i in range(n_engines)]

In [6]:
for d in wdirs:
    if not os.path.isdir(d):
        os.mkdir(d)

## `GmxEngine` is a thin wrapper around gromacs
- it has a `prepare()` method (which will call `grompp`) and multiple methods to then rujn the simulation, namely `run()`, `run_walltime()` and `run_nsteps()`

In [7]:
engines = [aimmdd.GmxEngine(gro_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                            top_file=os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                            ndx_file=os.path.join("gmx_infiles/index.ndx"),
                            mdrun_extra_args="-nt 2",  # use this if your version of GMX is compiled with thread-MPI support
                            #mdrun_extra_args="-ntomp 2",  # use this for GMX without thread-MPI support
                            )
           for _ in range(n_engines)]

In [8]:
mdps = [aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp"))
        for _ in range(n_engines)]

In [9]:
mdps[0].changed

False

In [10]:
for key, val in mdps[0].items():
    print(key, " : ", val)

title  :  ['test']
cpp  :  ['/lib/cpp']
include  :  ['-I../top']
define  :  []
integrator  :  ['md-vv']
dt  :  0.002
nsteps  :  100000
nstxout  :  10
nstvout  :  10
nstlog  :  10
nstenergy  :  10
nstxout-compressed  :  10
compressed-x-grps  :  ['Protein']
energygrps  :  ['Protein', 'SOL']
nstlist  :  10
ns-type  :  ['grid']
cutoff-scheme  :  ['Verlet']
rlist  :  1.1
coulombtype  :  ['PME']
rcoulomb  :  1.1
rvdw  :  1.1
tcoupl  :  ['Berendsen']
tc-grps  :  ['Protein', 'SOL']
tau-t  :  [0.1, 0.1]
ref-t  :  [300.0, 300.0]
Pcoupl  :  ['Berendsen']
tau-p  :  1.0
compressibility  :  [4.5e-05]
ref-p  :  [1.0]
gen-vel  :  ['no']
gen-temp  :  300.0
gen-seed  :  173529
constraints  :  ['all-bonds']


In [11]:
nsteps = 20
for mdp in mdps:
    mdp['nstvout'] = 20
    mdp["nstxout"] = 20
    mdp["nstxout-compressed"] = 5
    mdp["nstlist"] = 10
    mdp["continuation"] = "yes"

In [12]:
mdps[0]

{'title': ['test'], 'cpp': ['/lib/cpp'], 'include': ['-I../top'], 'define': [], 'integrator': ['md-vv'], 'dt': 0.002, 'nsteps': 100000, 'nstxout': 20, 'nstvout': 20, 'nstlog': 10, 'nstenergy': 10, 'nstxout-compressed': 5, 'compressed-x-grps': ['Protein'], 'energygrps': ['Protein', 'SOL'], 'nstlist': 10, 'ns-type': ['grid'], 'cutoff-scheme': ['Verlet'], 'rlist': 1.1, 'coulombtype': ['PME'], 'rcoulomb': 1.1, 'rvdw': 1.1, 'tcoupl': ['Berendsen'], 'tc-grps': ['Protein', 'SOL'], 'tau-t': [0.1, 0.1], 'ref-t': [300.0, 300.0], 'Pcoupl': ['Berendsen'], 'tau-p': 1.0, 'compressibility': [4.5e-05], 'ref-p': [1.0], 'gen-vel': ['no'], 'gen-temp': 300.0, 'gen-seed': 173529, 'constraints': ['all-bonds'], 'continuation': ['yes']}

In [13]:
mdps[0].changed

True

In [14]:
import time

In [15]:
from state_funcs_mda import descriptor_func_psi_phi

descriptor_func_wrap = aimmdd.PyTrajectoryFunctionWrapper(descriptor_func_psi_phi)#, {"scratch_dir": scratch_dir})

In [16]:
init_conf = aimmdd.Trajectory(trajectory_file=os.path.join("gmx_infiles/conf.trr"),
                              structure_file=os.path.join("gmx_infiles/conf.gro"),
                             )

init_conf2 = aimmdd.Trajectory(trajectory_file=os.path.join("gmx_infiles/other_conf.trr"),
                               structure_file=os.path.join("gmx_infiles/conf.gro"),
                              )

In [17]:
cv = await descriptor_func_wrap(init_conf)
print(cv)

[[0.90923707 1.49169309 0.50778891 1.08791043]]


In [18]:
cv = await descriptor_func_wrap(init_conf2)
print(cv)

[[1.00030301 0.50000009 0.67049795 0.62393033]]


In [19]:
descriptor_func_wrap_slurm = aimmdd.SlurmTrajectoryFunctionWrapper("state_funcs_mda.py", "gmx_infiles/calc_cvs_MASTER.slurm", call_kwargs={"--function": "descriptors"})

In [20]:
#await descriptor_func_wrap_slurm(init_conf)

FileNotFoundError: [Errno 2] No such file or directory: 'sbatch'

In [21]:
#await descriptor_func_wrap_slurm(init_conf2)

FileNotFoundError: [Errno 2] No such file or directory: 'sbatch'

In [22]:
start = time.time()
await asyncio.gather(*(e.prepare(starting_configuration=init_conf2, workdir=wdir, deffnm="blab", run_config=mdp)
                               for e, wdir, mdp in zip(engines, wdirs, mdps)
                               )
                            )
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.06507611274719238 s


In [23]:
walltime = 0.005 # 0.01 h = 36 s
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(walltime) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 18.64672541618347 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [24]:
cv = await descriptor_func_wrap(trajs[0])
cv

array([[1.00030301, 0.50000009, 0.67049795, 0.62393033],
       [1.04864427, 0.50237189, 0.70278706, 0.59792479],
       [1.01796867, 0.50032298, 0.71046427, 0.5923616 ],
       ...,
       [1.22999626, 0.5560386 , 0.53788412, 1.19091598],
       [1.1747439 , 0.53152954, 0.53712161, 1.18905977],
       [1.16050702, 0.52646278, 0.50753545, 1.08647931]])

In [25]:
cv.shape

(1061, 4)

In [26]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 1060 frames (== 21200 integration steps)
engine 1 did 1068 frames (== 21360 integration steps)
engine 2 did 1044 frames (== 20880 integration steps)
engine 3 did 1204 frames (== 24080 integration steps)


In [27]:
for t in trajs:
    print(len(t), t)

1061 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.tpr)
1069 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.tpr)
1045 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.tpr)
1205 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.part0001.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [28]:
walltime = 0.005 # 0.01 h = 36 s
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(walltime) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")
cv = await descriptor_func_wrap(trajs[0])
cv

time elapsed: 18.70518445968628 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


array([[1.16050702, 0.52646278, 0.50753545, 1.08647931],
       [1.17289006, 0.53084221, 0.50261395, 0.94894002],
       [1.18120339, 0.53398999, 0.50780033, 0.91202568],
       ...,
       [1.34222102, 0.63546636, 0.50018195, 0.98651235],
       [1.30967284, 0.60744079, 0.5042814 , 0.93470784],
       [1.30498082, 0.60378453, 0.51047094, 0.89820952]])

In [29]:
cv.shape

(1061, 4)

In [30]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 2120 frames (== 42400 integration steps)
engine 1 did 2124 frames (== 42480 integration steps)
engine 2 did 2100 frames (== 42000 integration steps)
engine 3 did 2264 frames (== 45280 integration steps)


In [31]:
for t in trajs:
    print(len(t), t)

1061 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.tpr)
1057 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.tpr)
1057 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.tpr)
1061 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.part0002.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [32]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(nsteps, steps_per_part=True) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.7660493850708008 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [33]:
cv = await descriptor_func_wrap(trajs[0])

In [34]:
cv

array([[1.30498082, 0.60378453, 0.51047094, 0.89820952],
       [1.29379113, 0.59541778, 0.62315692, 0.67138275]])

In [35]:
for t in trajs:
    print(len(t), t)

2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.part0003.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.tpr)
2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.part0003.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.tpr)
2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.part0003.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.tpr)
2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.part0003.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [36]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 2121 frames (== 42420 integration steps)
engine 1 did 2125 frames (== 42500 integration steps)
engine 2 did 2101 frames (== 42020 integration steps)
engine 3 did 2265 frames (== 45300 integration steps)


In [37]:
trajs[0].last_step

42420

In [38]:
u0 = mda.Universe(trajs[0].structure_file, trajs[0].trajectory_file)
ts00 = u0.trajectory[0]
print(ts00.__dict__)
#print(ts00.step)
ts0_1 = u0.trajectory[-1]
print(ts0_1.__dict__)
#print(ts0_1.step)

{'frame': 0, '_n_atoms': 1651, 'data': {'time': 84.80000305175781, 'step': 42400, 'lambda': 0.0, 'dt': 0.0399932861328125}, '_has_positions': True, '_has_velocities': True, '_has_forces': False, '_pos': array([[21.318285 , 22.719357 ,  7.1754274],
       [21.162472 , 21.656345 ,  7.3593864],
       [20.397308 , 23.2847   ,  7.3179197],
       ...,
       [15.161304 ,  3.5767522, 10.879899 ],
       [15.254726 ,  2.7780614, 10.360671 ],
       [15.658651 ,  4.236635 , 10.396751 ]], dtype=float32), '_unitcell': array([25.616795, 25.616795, 25.616795, 90.      , 90.      , 90.      ],
      dtype=float32), 'aux': {}, '_velocities': array([[  3.2818928,   4.9918995,  -2.7013025],
       [ 27.991894 ,   1.3098364,  -3.0318592],
       [  8.594737 ,  18.544773 , -22.111197 ],
       ...,
       [  1.3527086,   3.9159832,   9.509464 ],
       [ 13.512682 ,   3.5898223,  12.19906  ],
       [  1.3556803,   6.9213963,  13.617302 ]], dtype=float32)}
{'frame': 1, '_n_atoms': 1651, 'data': {'time'

In [39]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(nsteps, steps_per_part=True) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.7198045253753662 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [40]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[1.29379112 0.59541778 0.62315683 0.67138285]
 [1.30179225 0.60135049 0.66993855 0.6244212 ]]


In [41]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 2122 frames (== 42440 integration steps)
engine 1 did 2126 frames (== 42520 integration steps)
engine 2 did 2102 frames (== 42040 integration steps)
engine 3 did 2266 frames (== 45320 integration steps)


In [42]:
for t in trajs:
    print(len(t), t)

2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.part0004.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.tpr)
2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.part0004.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.tpr)
2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.part0004.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.tpr)
2 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.part0004.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.tpr)


In [43]:
trajs[0].first_step

42420

In [44]:
u0 = mda.Universe(trajs[0].structure_file, trajs[0].trajectory_file)
ts00 = u0.trajectory[0]
print(ts00.__dict__)
ts0_1 = u0.trajectory[-1]
print(ts0_1.__dict__)

{'frame': 0, '_n_atoms': 1651, 'data': {'time': 84.83999633789062, 'step': 42420, 'lambda': 0.0, 'dt': 0.04000091552734375}, '_has_positions': True, '_has_velocities': True, '_has_forces': False, '_pos': array([[21.315407 , 22.863338 ,  7.013219 ],
       [20.97101  , 21.859734 ,  7.2627907],
       [20.434217 , 23.443325 ,  7.287519 ],
       ...,
       [15.306646 ,  3.6804924, 11.239339 ],
       [15.083027 ,  3.140565 , 10.481247 ],
       [15.786357 ,  4.4229364, 10.87207  ]], dtype=float32), '_unitcell': array([25.615911, 25.615911, 25.615911, 90.      , 90.      , 90.      ],
      dtype=float32), 'aux': {}, '_velocities': array([[  0.5970922 ,   3.2202115 ,  -5.7903905 ],
       [  3.1636655 ,  -0.16787097, -15.873276  ],
       [ -4.9285374 ,  -6.3846664 ,  -3.2326386 ],
       ...,
       [  5.940548  ,   3.1056645 ,   5.3127246 ],
       [-16.627136  ,  -8.9873085 ,  20.582512  ],
       [ -0.0311041 ,  -5.6588907 , -20.20504   ]], dtype=float32)}
{'frame': 1, '_n_atoms': 16

In [45]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.001) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 4.336025238037109 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [46]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[1.30179225 0.60135049 0.66993855 0.6244212 ]
 [1.34446865 0.63758952 0.69121041 0.60674564]
 [1.22284887 0.55240824 0.6834174  0.61299166]
 [1.22265413 0.55231134 0.59833466 0.70223339]
 [1.20659522 0.54467768 0.57710458 0.73324269]
 [1.25496574 0.56989249 0.53831009 0.80805618]
 [1.24513418 0.56421423 0.52297252 0.85018417]
 [1.25324301 0.56887592 0.50380061 0.93846818]
 [1.27775807 0.58424713 0.50109255 0.96696424]
 [1.26389101 0.57531007 0.5017449  0.95826442]
 [1.35688078 0.64980562 0.51717418 1.12992008]
 [1.24684334 0.56518007 0.50897988 1.0943358 ]
 [1.25860657 0.57207169 0.55802324 1.23378739]
 [1.28615747 0.58998304 0.53824112 1.19177783]
 [1.31022543 0.60787734 0.54462502 1.20647911]
 [1.30962768 0.60740517 0.51178157 1.10790164]
 [1.28076849 0.58627418 0.55874053 1.23513844]
 [1.22554001 0.55375824 0.53892426 1.19341448]
 [1.17430393 0.53136567 0.51195824 1.10869793]
 [1.07587143 0.50579    0.52447128 1.15450709]
 [1.12553278 0.51601496 0.51384596 1.1168514 ]
 [1.17538862 

In [47]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 2336 frames (== 46720 integration steps)
engine 1 did 2340 frames (== 46800 integration steps)
engine 2 did 2316 frames (== 46320 integration steps)
engine 3 did 2480 frames (== 49600 integration steps)


In [48]:
trajs[0].first_step

42440

In [49]:
u0 = mda.Universe(trajs[0].structure_file, trajs[0].trajectory_file)
ts00 = u0.trajectory[0]
print(ts00.__dict__)
ts0_1 = u0.trajectory[-1]
print(ts0_1.__dict__)

{'frame': 0, '_n_atoms': 1651, 'data': {'time': 84.87999725341797, 'step': 42440, 'lambda': 0.0, 'dt': 0.04000091552734375}, '_has_positions': True, '_has_velocities': True, '_has_forces': False, '_pos': array([[21.477219 , 22.931625 ,  6.8208265],
       [21.328547 , 21.856281 ,  6.7226124],
       [20.550337 , 23.453512 ,  7.058826 ],
       ...,
       [15.293292 ,  3.5782528, 11.262589 ],
       [15.421253 ,  3.001536 , 10.509426 ],
       [15.746075 ,  4.387409 , 11.024924 ]], dtype=float32), '_unitcell': array([25.622042, 25.622042, 25.622042, 90.      , 90.      , 90.      ],
      dtype=float32), 'aux': {}, '_velocities': array([[  2.0306478 ,  -0.91754556,  -4.1496267 ],
       [ -5.63541   ,  -0.35114384,   1.3034946 ],
       [ -2.8269694 , -16.671331  ,  11.456746  ],
       ...,
       [ -3.3717937 ,  -5.314746  ,  -2.4013093 ],
       [  4.878375  ,   0.4814887 ,  -5.437949  ],
       [ -7.297474  ,  -0.7690326 ,   5.5961204 ]], dtype=float32)}
{'frame': 214, '_n_atoms': 

In [50]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.001) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

cv = await descriptor_func_wrap(trajs[0])
print(cv)


time elapsed: 4.382994174957275 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


[[1.36670132 0.66010274 0.51621788 1.12631253]
 [1.40216427 0.70290759 0.51672863 1.12825282]
 [1.35787349 0.65082016 0.51010322 1.1000057 ]
 [1.42506845 0.73671914 0.50837636 0.90886167]
 [1.42471145 0.73614363 0.50349897 0.94095146]
 [1.43676042 0.75660662 0.50560019 1.07462461]
 [1.40559302 0.70760592 0.50748943 1.08621681]
 [1.46072587 0.80575357 0.50881858 1.0934923 ]
 [1.4864703  0.88447231 0.51921201 1.13726949]
 [1.49529984 0.93160358 0.55297187 1.22397734]
 [1.49803663 0.95573361 0.55767819 1.23313389]
 [1.49466111 0.92712759 0.5567708  1.23140414]
 [1.48587229 0.88198256 0.53577711 1.18573397]
 [1.48800256 0.89112622 0.57911532 1.26991867]
 [1.45552758 0.79385776 0.52744546 1.16337751]
 [1.35930949 0.65229798 0.51200928 1.10892686]
 [1.28542119 0.58947017 0.51537643 1.12304467]
 [1.38336906 0.67901999 0.51387224 1.11696069]
 [1.44717027 0.77630657 0.55010778 1.21816735]
 [1.44522723 0.77246383 0.513967   1.11735382]
 [1.42232966 0.73234789 0.52715318 1.16252963]
 [1.39765692 

In [51]:
trajs

[Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.part0006.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir0/blab.tpr),
 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.part0006.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir1/blab.tpr),
 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.part0006.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir2/blab.tpr),
 Trajectory(trajectory_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.part0006.trr, structure_file=/home/think/scratch/aimmd_distributed/engine_wdir3/blab.tpr)]

In [52]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(20, steps_per_part=True) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.7138702869415283 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [53]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[1.24571594 0.56454199 0.52273334 0.85094787]
 [1.28514451 0.58927795 0.5491077  0.78390711]]


In [56]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(20, steps_per_part=True) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")
cv = await descriptor_func_wrap(trajs[0])
print(cv)

time elapsed: 0.6843156814575195 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


[[1.28514451 0.58927795 0.5491077  0.78390711]
 [1.23435407 0.55832345 0.5257264  0.84168211]]


In [38]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(1) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")
cv = await descriptor_func_wrap(trajs[0])
print(cv)

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


OSError: XDR read error = integer

Exception ignored in: <function ReaderBase.__del__ at 0x7f0aefa608b0>
Traceback (most recent call last):
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py", line 2152, in __del__
    self.close()
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py", line 179, in close
    self._xdr.close()
AttributeError: 'TRRReader' object has no attribute '_xdr'
Exception ignored in: <function ReaderBase.__del__ at 0x7f0aefa608b0>
Traceback (most recent call last):
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py", line 2152, in __del__
    self.close()
  File "/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py", line 179, in close
    self._xdr.close()
AttributeError: 'TRRReader' object has no attribute '_xdr'
Exception ignored in: <function ReaderBase

In [36]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9222 frames (== 92220 integration steps)
engine 1 did 9206 frames (== 92060 integration steps)
engine 2 did 9006 frames (== 90060 integration steps)
engine 3 did 9046 frames (== 90460 integration steps)


In [29]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.01) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 36.5525848865509 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [30]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[ 1.7949889 -2.3867085]
 [ 1.7086705 -2.3630643]
 [ 1.7360168 -2.3054514]
 ...
 [-1.4733312 -1.5860335]
 [-1.4640243 -1.643299 ]
 [-1.4391651 -1.8763256]]


In [31]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 12460 frames (== 124600 integration steps)
engine 1 did 12700 frames (== 127000 integration steps)
engine 2 did 12494 frames (== 124940 integration steps)
engine 3 did 13316 frames (== 133160 integration steps)


In [32]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.01) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 36.527894735336304 s


/home/think/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [33]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[-1.4391651 -1.8763256]
 [-1.4037466 -1.9764206]
 [-1.4453897 -1.972756 ]
 ...
 [ 1.3600926 -2.4688985]
 [ 1.3947617 -2.3246064]
 [ 1.5066161 -2.267869 ]]


In [34]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 16605 frames (== 166050 integration steps)
engine 1 did 16869 frames (== 168690 integration steps)
engine 2 did 16661 frames (== 166610 integration steps)
engine 3 did 17440 frames (== 174400 integration steps)


# propagator until state

In [13]:
import mdtraj as mdt
import numpy as np
"""
def psi(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_psi(traj)
    return psi

def alpha_R(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_dihedrals(traj, indices=[[6,8,14,16]])[:, 0]
    phi = mdt.compute_dihedrals(traj, indices=[[4,6,8,14]])[:, 0]
    state = np.full_like(psi, False, dtype=bool)
    # phi: -pi -> 0 
    # psi: > -50 but smaller 30 degree
    deg = 180/np.pi
    state[(phi <= 0) & (-50/deg <= psi) & (psi <= 30/deg)] = True
    return state

wrapped_alphaR = arcdd.trajectory.TrajectoryFunctionWrapper(alpha_R)
wrapped_psi = arcdd.trajectory.TrajectoryFunctionWrapper(psi)
"""

from state_funcs import alpha_R, C7_eq
wrapped_alphaR = aimmdd.TrajectoryFunctionWrapper(alpha_R, call_kwargs={"scratch_dir": scratch_dir})
wrapped_C7_eq = aimmdd.TrajectoryFunctionWrapper(C7_eq, call_kwargs={"scratch_dir": scratch_dir})

In [14]:
import aimmd.distributed.logic

In [15]:
propagators = [aimmd.distributed.logic.PropagatorUntilAnyState(states=[wrapped_alphaR, wrapped_C7_eq],
                                                               engine_cls=aimmdd.GmxEngine,
                                                               engine_kwargs={"gro_file":os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                                                              "top_file":os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                                                                               "mdrun_extra_args": "-nt 4",
                                                                              },
                                                               run_config=aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp")),
                                                               walltime_per_part=0.005,
                                                               
                                                              )
               for _ in range(n_engines)
               ]

Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.


In [17]:
#initial_tp_mdt = mdt.load(os.path.join(scratch_dir, "gmx_infiles/ala_400K_TP_low_barrier.h5"))

In [18]:
#initial_tp_mdt[10].save_trr(os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"))

In [16]:
starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro"))

starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

In [17]:
wrapped_alphaR

TrajectoryFunctionWrapper(function=<function alpha_R at 0x7fe1fe04e700>, call_kwargs={'scratch_dir': '/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'})

In [18]:
import mdtraj as mdt

In [19]:
mdt.load(os.path.join(scratch_dir, "gmx_infiles/conf.gro"), top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (ALA, ACE)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (NME, ALA)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (HOH, NME)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/core/trajectory.py:422: UserWarning: top= kwarg igno

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7fe126c51d30>

In [20]:
mdt.load(starting_conf.trajectory_file, top=starting_conf.structure_file)

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7fe126bff550>

In [21]:
alpha_R(starting_conf, **{"scratch_dir": scratch_dir})

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


array([ True])

In [22]:
await wrapped_alphaR(starting_conf)

array([ True])

In [23]:
scratch_dir

'/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'

In [24]:
tasks = [asyncio.create_task(p.propagate_and_concatenate(starting_configuration=starting_conf,
                                                         workdir=wdir,
                                                         deffnm="blib",
                                                         tra_out=os.path.join(wdir, "blub_until_state.trr"),
                                                          ))
         for p, wdir, mdp in zip(propagators, wdirs, mdps)]

done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)

In [25]:
list(done)[0]

<Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>

In [26]:
tasks

[<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() 

In [27]:
done, pending = await asyncio.wait(tasks,)# return_when=asyncio.FIRST_COMPLETED)

In [28]:
for i in done:
    print(i)
    print(tasks.index(i))

<Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
2
<Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
3
<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
0
<Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() g

In [29]:
pending

set()

In [21]:
start = time.time()
result1 = await wrapped_func(trajs[-1])
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.48328495025634766 s


In [22]:
start = time.time()
results = await asyncio.gather(*(wrapped_func(t) for t in trajs))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.5739212036132812 s


In [23]:
trajs

[Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir5/blub.part0002.trr, topology_file=/home/think/scratch/arcd_dist